# 任务一
## 在投影坐标系下，完成椭圆曲线上点的计算：2𝐺，𝐺+(2𝐺)，31𝐺

```python
# P-256 参数
p = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF
a = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
G = (0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296,
     0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5)
n = 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551 #（阶）
```

### 一、定义参数与辅助函数

In [1]:
p = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF
a = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5
n = 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551
G = (Gx, Gy)
INF = (0, 1, 0)

def mod_pow(x, y, m):
    """计算 (x^y) % m"""
    return pow(x, y, m)

def extended_gcd(a, b):
    """扩展欧几里得算法"""
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = extended_gcd(b % a, a)
        return (g, y - (b // a) * x, x)

def mod_inverse(a, m):
    """计算模反元素，即找到b使得(a*b) % m == 1"""
    g, x, y = extended_gcd(a, m)
    if g != 1:
        if a % m == 0:
            return 0 
        else:
            raise Exception(f'模反元素不存在: gcd({a}, {m}) = {g}')
    else:
        return x % m

### 二、仿射坐标系（ACS）下，普通计算方式与蒙哥马利模乘计算方式的实现

1. 仿射坐标系（ACS）下，普通计算方式的实现

In [2]:
def point_add(P, Q):
    """椭圆曲线上的点加法"""
    if P is None:
        return Q
    if Q is None:
        return P
    
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 != y2:
        return None
    
    if P == Q:
        denominator = (2 * y1) % p
        if denominator == 0:
            return None  
        s = (3 * x1**2 + a) * mod_inverse(denominator, p) % p
    else:
        denominator = (x2 - x1) % p
        if denominator == 0:
            return None 
        s = (y2 - y1) * mod_inverse(denominator, p) % p
    x3 = (s**2 - x1 - x2) % p
    y3 = (s * (x1 - x3) - y1) % p
    return (x3, y3)

def point_mul(P, k):
    """椭圆曲线上的点乘法"""
    result = None
    addend = P
    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1
    return result

def compute_points_normal():
    _2G = point_add(G, G)
    _3G = point_add(G, _2G)
    _31G = point_mul(G, 31)
    return _2G, _3G, _31G

In [3]:
print("============ 普通计算方式计算结果 ============")
_2G_a, _3G_a, _31G_a = compute_points_normal()
print(f"2G = ({hex(_2G_a[0])}, {hex(_2G_a[1])})")
print(f"G + (2G) = ({hex(_3G_a[0])}, {hex(_3G_a[1])})")
print(f"31G = ({hex(_31G_a[0])}, {hex(_31G_a[1])})")

============ 普通计算方式计算结果 ============
2G = (0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978, 0x7775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1)
G + (2G) = (0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c, 0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384fb83d9a79b127a27d5032)
31G = (0x301d9e502dc7e05da85da026a7ae9aa0fac9db7d52a95b3e3e3f9aa0a1b45b8b, 0x6551b6f6b3061223e0d23c026b017d72298d9ae46887ca61d58db6aea17ee267)


2. 仿射坐标系（ACS）下，蒙哥马利模乘计算方式的实现

In [4]:
class Montgomery_ACS:
    def __init__(self, MOD):
        self.MOD = MOD
        self.R = 1 << MOD.bit_length()
        self.INV_R = mod_inverse(self.R, MOD)
        _, inv, _ = extended_gcd(MOD, self.R)
        self.MOD_INV = (self.R - (inv % self.R)) % self.R
    
    def convert_to_mont(self, x):
        """将普通数转换为蒙哥马利形式"""
        return (x * self.R) % self.MOD
    
    def convert_from_mont(self, x):
        """将蒙哥马利形式转换为普通数"""
        return (x * self.INV_R) % self.MOD
    
    def mul(self, a, b):
        """蒙哥马利模乘 [a * b * (R^-1)] mod m"""
        multiply = a * b
        temp = (multiply * self.MOD_INV) & (self.R - 1)
        res = (multiply + temp * self.MOD) >> self.MOD.bit_length()
        return res % self.MOD
    
    def add(self, a, b):
        """蒙哥马利域中的加法 a + b mod m"""
        return (a + b) % self.MOD
    
    def sub(self, a, b):
        """蒙哥马利域中的减法 a - b mod m"""
        return (a - b) % self.MOD

def point_add_mont(P, Q, a_mont, mont):
    """使用蒙哥马利模乘的点加运算"""
    if P is None:
        return Q
    if Q is None:
        return P
    
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 != y2:
        return None
    
    if x1 == x2:
        x1_squared = mont.mul(x1, x1)
        three = mont.convert_to_mont(3)
        three__x1_squared = mont.mul(three, x1_squared)
        numerator = mont.add(three__x1_squared, a_mont)
        two = mont.convert_to_mont(2)
        two__y1 = mont.mul(two, y1)

        denominator = mont.convert_from_mont(two__y1)
        inv_denominator = mod_inverse(denominator, mont.MOD)
        inv_denominator_mont = mont.convert_to_mont(inv_denominator)
        
        lambda_val = mont.mul(numerator, inv_denominator_mont)
    else: 
        y_d = mont.sub(y2, y1)
        x_d = mont.sub(x2, x1)
        x_d_normal = mont.convert_from_mont(x_d)
        inv_x_d = mod_inverse(x_d_normal, mont.MOD)
        inv_x_d_mont = mont.convert_to_mont(inv_x_d)

        lambda_val = mont.mul(y_d, inv_x_d_mont)
    
    lambda_squared = mont.mul(lambda_val, lambda_val)
    x_sum = mont.add(x1, x2)
    x3 = mont.sub(lambda_squared, x_sum)
    
    x1_sub_x3 = mont.sub(x1, x3)
    lambda_x_d = mont.mul(lambda_val, x1_sub_x3)
    y3 = mont.sub(lambda_x_d, y1)
    
    return (x3, y3)

def point_multiply_mont(k, P, a_mont, mont):
    """使用蒙哥马利模乘的点倍运算 kP"""
    res = None
    ad = P
    while k > 0:
        if k & 1:
            res = point_add_mont(res, ad, a_mont, mont)
        ad = point_add_mont(ad, ad, a_mont, mont)
        k >>= 1 
    return res

def compute_points_with_mont():
    mont = Montgomery_ACS(p)
    
    a_mont = mont.convert_to_mont(a)
    Gx_mont = mont.convert_to_mont(Gx)
    Gy_mont = mont.convert_to_mont(Gy)
    G_mont = (Gx_mont, Gy_mont)
    
    res_2G_mont = point_multiply_mont(2, G_mont, a_mont, mont)
    res_2G = (mont.convert_from_mont(res_2G_mont[0]), 
                mont.convert_from_mont(res_2G_mont[1]))
    
    res_G_add_2G_mont = point_add_mont(G_mont, res_2G_mont, a_mont, mont)
    res_G_add_2G = (mont.convert_from_mont(res_G_add_2G_mont[0]), 
                        mont.convert_from_mont(res_G_add_2G_mont[1]))
   

    res_31G_mont = point_multiply_mont(31, G_mont, a_mont, mont)
    res_31G = (mont.convert_from_mont(res_31G_mont[0]), 
                 mont.convert_from_mont(res_31G_mont[1]))
    
    return res_2G, res_G_add_2G, res_31G

In [5]:
print("============ 蒙哥马利模乘计算方式计算结果 ============")
res_2G_a, res_3G_a, res_31G_a = compute_points_with_mont()
print(f"2G = ({hex(res_2G_a[0])}, {hex(res_2G_a[1])})")
print(f"G + (2G) = ({hex(res_3G_a[0])}, {hex(res_3G_a[1])})")
print(f"31G = ({hex(res_31G_a[0])}, {hex(res_31G_a[1])})")

============ 蒙哥马利模乘计算方式计算结果 ============
2G = (0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978, 0x7775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1)
G + (2G) = (0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c, 0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384fb83d9a79b127a27d5032)
31G = (0x301d9e502dc7e05da85da026a7ae9aa0fac9db7d52a95b3e3e3f9aa0a1b45b8b, 0x6551b6f6b3061223e0d23c026b017d72298d9ae46887ca61d58db6aea17ee267)


### 三、（雅可比）投影坐标系（Jacobi PCS）下，普通计算方式与蒙哥马利模乘计算方式的实现

1. （雅可比）投影坐标系（Jacobi PCS）下，普通计算方式的实现

In [6]:
def point_add_jacobian(P, Q):
    """雅可比坐标系下的点加法 P + Q"""
    if P == INF:
        return Q
    if Q == INF:
        return P
    X1, Y1, Z1 = P
    X2, Y2, Z2 = Q
    if (X1 * mod_pow(Z2, 2, p) - X2 * mod_pow(Z1, 2, p)) % p == 0:
        if (Y1 * mod_pow(Z2, 3, p) + Y2 * mod_pow(Z1, 3, p)) % p == 0:
            return INF
    Z1Z1 = (Z1 * Z1) % p
    Z2Z2 = (Z2 * Z2) % p
    U1 = (X1 * Z2Z2) % p
    U2 = (X2 * Z1Z1) % p
    S1 = (Y1 * Z2 * Z2Z2) % p
    S2 = (Y2 * Z1 * Z1Z1) % p
    if U1 == U2:
        if S1 != S2:
            return INF
        return point_double_jacobian(P)  
    H = (U2 - U1) % p
    R = (S2 - S1) % p
    H2 = (H * H) % p
    H3 = (H * H2) % p
    V = (U1 * H2) % p
    X3 = (R * R - H3 - 2 * V) % p
    Y3 = (R * (V - X3) - S1 * H3) % p
    Z3 = (Z1 * Z2 * H) % p
    return (X3, Y3, Z3)

def point_double_jacobian(P):
    """雅可比坐标系下的点倍乘 2P"""
    if P == INF:
        return INF
    X, Y, Z = P
    if Y == 0:
        return INF
    Y2 = (Y * Y) % p
    Y4 = (Y2 * Y2) % p
    S = (4 * X * Y2) % p
    M = (3 * X * X + a * Z * Z * Z * Z) % p
    X3 = (M * M - 2 * S) % p
    Y3 = (M * (S - X3) - 8 * Y4) % p
    Z3 = (2 * Y * Z) % p
    return (X3, Y3, Z3)

def point_mul_jacobian(P, k):
    """雅可比坐标系下的点乘法 kP"""
    if k == 0:
        return INF
    result = INF
    addend = P
    while k > 0:
        if k & 1:
            result = point_add_jacobian(result, addend)
        addend = point_double_jacobian(addend)
        k >>= 1
    return result

def jacobian_to_affine(P):
    """将雅可比坐标转换为仿射坐标"""
    if P == INF:
        return None  
    X, Y, Z = P
    Z_inv = mod_inverse(Z, p)
    Z2_inv = (Z_inv * Z_inv) % p
    Z3_inv = (Z2_inv * Z_inv) % p
    x = (X * Z2_inv) % p
    y = (Y * Z3_inv) % p
    return (x, y)

def compute_points_projective():
    G_jacobian = (Gx, Gy, 1)
    _2G_jacobian = point_double_jacobian(G_jacobian)
    _2G = jacobian_to_affine(_2G_jacobian)
    _3G_jacobian = point_add_jacobian(G_jacobian, _2G_jacobian)
    _3G = jacobian_to_affine(_3G_jacobian)
    _31G_jacobian = point_mul_jacobian(G_jacobian, 31)
    _31G = jacobian_to_affine(_31G_jacobian)
    return _2G, _3G, _31G, _2G_jacobian, _3G_jacobian, _31G_jacobian

In [7]:
print("============ 普通计算方式计算结果 ============\n")
_2G_p, _3G_p, _31G_p, _2G_jacobian_p, _3G_jacobian_p, _31G_jacobian_p = compute_points_projective()
print("======== 投影坐标系的结果 ========")
print(f"2G = ({hex(_2G_jacobian_p[0])}, {hex(_2G_jacobian_p[1])}, {hex(_2G_jacobian_p[2])})")
print(f"G + (2G) = ({hex(_3G_jacobian_p[0])}, {hex(_3G_jacobian_p[1])}, {hex(_3G_jacobian_p[2])})")
print(f"31G = ({hex(_31G_jacobian_p[0])}, {hex(_31G_jacobian_p[1])}, {hex(_31G_jacobian_p[2])})")
print()
print("======== 转成仿射坐标系后的结果 ========")
print(f"2G = ({hex(_2G_p[0])}, {hex(_2G_p[1])})")
print(f"G + (2G) = ({hex(_3G_p[0])}, {hex(_3G_p[1])})")
print(f"31G = ({hex(_31G_p[0])}, {hex(_31G_p[1])})")

============ 普通计算方式计算结果 ============

======== 投影坐标系的结果 ========
2G = (0x9a978f59acd1b5ad570e7d52dcfcde43804b42274f61ddcf1e7d848391d6c70f, 0x4126885e7f786af905338238e5346d5fe77fc46388668bd0fd59be3190d2f5d1, 0x9fc685c5fc34ff371dcfd694f81f3c2c579c66aed662bd9d976c80d06f7ea3ea)
G + (2G) = (0xec88440c8b00a9e572bf1bceb7d0c5906bd65990a9b7081130bd72e2c136ca0, 0x3f43881dcc61766ffe07171330863c080b4422e506909bb9e6642bdcfd959f29, 0x22d834ac5ce2e092fbe5330eefc21fa45f5a1229b48c72ae01cc6821f1c03cf9)
31G = (0x2308c580be979559af21da942a396e634676dc1ee604f27029bb20479fdd496b, 0xde157146d19f8efbd8eb9c5b3ccc8646e62e7c490a1110a1d02127900854721f, 0xc39f3be4cb3e64a988342896fcc835da0b9593d6158ecc8fecfaa7fdc9e2c5f0)

======== 转成仿射坐标系后的结果 ========
2G = (0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978, 0x7775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1)
G + (2G) = (0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c, 0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384fb

2. （雅可比）投影坐标系（Jacobi PCS）下，蒙哥马利模乘计算方式的实现

In [8]:
class Montgomery_PCS:
    def __init__(self, MOD):
        self.MOD = MOD
        self.R = 1 << MOD.bit_length()
        self.INV_R = mod_inverse(self.R, MOD)
        _, inv, _ = extended_gcd(MOD, self.R)
        self.MOD_INV = (self.R - (inv % self.R)) % self.R
    
    def convert_to_mont(self, x):
        """将普通数转换为蒙哥马利形式"""
        return (x * self.R) % self.MOD
    
    def convert_from_mont(self, x):
        """将蒙哥马利形式转换为普通数"""
        return (x * self.INV_R) % self.MOD
    
    def mul(self, a, b):
        """蒙哥马利模乘 [a * b * (R^-1)] mod m"""
        multiply = a * b
        temp = (multiply * self.MOD_INV) & (self.R - 1)
        res = (multiply + temp * self.MOD) >> self.MOD.bit_length()
        return res % self.MOD
    
    def add(self, a, b):
        """蒙哥马利域中的加法 a + b mod m"""
        return (a + b) % self.MOD
    
    def sub(self, a, b):
        """蒙哥马利域中的减法 a - b mod m"""
        return (a - b) % self.MOD

def point_add_mont_jacobian(P, Q, a_mont, mont):
    """雅可比坐标系下使用蒙哥马利模乘的点加法 P + Q"""
    if P == INF:
        return Q
    if Q == INF:
        return P
    
    X1, Y1, Z1 = P
    X2, Y2, Z2 = Q
    Z1Z1 = mont.mul(Z1, Z1)
    Z2Z2 = mont.mul(Z2, Z2)
    U1 = mont.mul(X1, Z2Z2)
    U2 = mont.mul(X2, Z1Z1)
    Z2_cubed = mont.mul(Z2, Z2Z2)
    Z1_cubed = mont.mul(Z1, Z1Z1)
    S1 = mont.mul(Y1, Z2_cubed)
    S2 = mont.mul(Y2, Z1_cubed)
    
    if U1 == U2:
        if S1 == mont.sub(0, S2):
            return INF
    if U1 == U2 and S1 == S2:
        return point_double_mont_jacobian(P, a_mont, mont)
    
    H = mont.sub(U2, U1)
    R = mont.sub(S2, S1)
    H2 = mont.mul(H, H)
    H3 = mont.mul(H, H2)
    V = mont.mul(U1, H2)
    X3 = mont.sub(mont.sub(mont.mul(R, R), H3), mont.mul(V, mont.convert_to_mont(2)))
    Y3 = mont.sub(mont.mul(R, mont.sub(V, X3)), mont.mul(S1, H3))
    Z3 = mont.mul(mont.mul(Z1, Z2), H)
    return (X3, Y3, Z3)

def point_double_mont_jacobian(P, a_mont, mont):
    """雅可比坐标系下使用蒙哥马利模乘的点倍乘 2P"""
    if P == INF:
        return INF
    
    X, Y, Z = P

    if Y == 0:
        return INF
    
    Y2 = mont.mul(Y, Y)
    Y4 = mont.mul(Y2, Y2)
    four = mont.convert_to_mont(4)
    S = mont.mul(four, mont.mul(X, Y2))
    three = mont.convert_to_mont(3)
    X_squared = mont.mul(X, X)
    three_X_squared = mont.mul(three, X_squared)
    Z_squared = mont.mul(Z, Z)
    Z_fourth = mont.mul(Z_squared, Z_squared)
    a_Z_fourth = mont.mul(a_mont, Z_fourth)
    
    M = mont.add(three_X_squared, a_Z_fourth)
    
    X3 = mont.sub(mont.mul(M, M), mont.mul(S, mont.convert_to_mont(2)))
    Y3 = mont.sub(mont.mul(M, mont.sub(S, X3)), mont.mul(mont.convert_to_mont(8), Y4))
    Z3 = mont.mul(mont.convert_to_mont(2), mont.mul(Y, Z))
    return (X3, Y3, Z3)

def point_mul_mont_jacobian(k, P, a_mont, mont):
    """雅可比坐标系下使用蒙哥马利模乘的点乘法 kP"""
    if k == 0:
        return INF
    result = INF
    addend = P
    while k > 0:
        if k & 1:
            result = point_add_mont_jacobian(result, addend, a_mont, mont)
        addend = point_double_mont_jacobian(addend, a_mont, mont)
        k >>= 1
    return result

def mont_jacobian_to_affine(P, mont):
    """将蒙哥马利雅可比坐标转换为普通仿射坐标"""
    if P == INF:
        return None
    
    X, Y, Z = P
    Z_normal = mont.convert_from_mont(Z)
    Z_inv = mod_inverse(Z_normal, mont.MOD)
    Z2_inv = (Z_inv * Z_inv) % mont.MOD
    Z3_inv = (Z2_inv * Z_inv) % mont.MOD
    x = (mont.convert_from_mont(X) * Z2_inv) % mont.MOD
    y = (mont.convert_from_mont(Y) * Z3_inv) % mont.MOD
    return (x, y)

def compute_points_mont_jacobian():
    mont = Montgomery_PCS(p)
    a_mont = mont.convert_to_mont(a)
    Gx_mont = mont.convert_to_mont(Gx)
    Gy_mont = mont.convert_to_mont(Gy)
    Z_mont = mont.convert_to_mont(1)
    G_mont_jacobian = (Gx_mont, Gy_mont, Z_mont)
    _2G_mont_jacobian = point_double_mont_jacobian(G_mont_jacobian, a_mont, mont)
    _2G = mont_jacobian_to_affine(_2G_mont_jacobian, mont)
    _3G_mont_jacobian = point_add_mont_jacobian(G_mont_jacobian, _2G_mont_jacobian, a_mont, mont)
    _3G = mont_jacobian_to_affine(_3G_mont_jacobian, mont)
    _31G_mont_jacobian = point_mul_mont_jacobian(31, G_mont_jacobian, a_mont, mont)
    _31G = mont_jacobian_to_affine(_31G_mont_jacobian, mont)
    return _2G, _3G, _31G, _2G_mont_jacobian, _3G_mont_jacobian, _31G_mont_jacobian


In [9]:
print("============ 蒙哥马利模乘计算方式计算结果 ============\n")
res_2G_p, res_3G_p, res_31G_p, res_2G_jacobian_p, res_3G_jacobian_p, res_31G_jacobian_p = compute_points_mont_jacobian()
print("======== 投影坐标系的结果 ========")
print(f"2G = ({hex(res_2G_jacobian_p[0])}, {hex(res_2G_jacobian_p[1])}, {hex(res_2G_jacobian_p[2])})")
print(f"G + (2G) = ({hex(res_3G_jacobian_p[0])}, {hex(res_3G_jacobian_p[1])}, {hex(res_3G_jacobian_p[2])})")
print(f"31G = ({hex(res_31G_jacobian_p[0])}, {hex(res_31G_jacobian_p[1])}, {hex(res_31G_jacobian_p[2])})")
print()
print("======== 转成仿射坐标系后的结果 ========")
print(f"2G = ({hex(res_2G_p[0])}, {hex(res_2G_p[1])})")
print(f"G + (2G) = ({hex(res_3G_p[0])}, {hex(res_3G_p[1])})")
print(f"31G = ({hex(res_31G_p[0])}, {hex(res_31G_p[1])})")

============ 蒙哥马利模乘计算方式计算结果 ============

======== 投影坐标系的结果 ========
2G = (0x5de4f991e25e717ad2aad563a7090a86c6b2e33a31355d9bf26ac1b54f2bf829, 0xf9f84f1f77bd1fdeda98128dce880027cf48a95eaa13daa3dab967123ccc950d, 0xae3fe314b10bb0aa5d10d11ba43e64b169571c87433c8b9bbe4a6af9d2aac15)
G + (2G) = (0x46b25328fadd64b3d27a6dd305abf1aedb90d91e2b4557e0385baba2d08d581, 0x7afe25c8fdb8bf98c23fe4853c87550f525e81c90828a8fdb2c275edddc6de96, 0x5abb037364b19905c081d7187dd8ab2983b8bce398463c894af672dd4ba85fab)
31G = (0x32082676761bbefd1ffd6cd618e1490a3e140649bb9992054c46ee5eb2eaf43, 0x9c9d166a317f922c42ec3b0a22dd9c0d68569682679ef04c14c5154d0aef6bca, 0x6e9fbacaba50606e495ff5ffd2adacfd1e7312220a12973feeaf758e5b0d16ff)

======== 转成仿射坐标系后的结果 ========
2G = (0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978, 0x7775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1)
G + (2G) = (0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c, 0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384

**<span style="color:red">注：普通计算方式和蒙哥马利模乘计算方式得到的投影坐标系不一样是正常的，因为其不唯一和蒙哥马利因子R的影响。</span>**

### 四、使用标准库（ecdsa 和 cryptography）计算结果

In [10]:
import ecdsa
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.backends import default_backend

def verify_with_ecdsa():
    curve = ecdsa.ellipticcurve.CurveFp(p, a, b)
    generator = ecdsa.ellipticcurve.Point(curve, Gx, Gy, n)
    two_G = generator * 2
    three_G = generator + two_G
    thirty_one_G = generator * 31
    return {
        "2G": (two_G.x(), two_G.y()),
        "G + (2G)": (three_G.x(), three_G.y()),
        "31G": (thirty_one_G.x(), thirty_one_G.y())
    }

def verify_with_cryptography():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_numbers = private_key.public_key().public_numbers()
    base_point = (public_numbers.x, public_numbers.y)
    private_key_2 = ec.derive_private_key(2, ec.SECP256R1(), default_backend())
    two_G = private_key_2.public_key().public_numbers()
    private_key_3 = ec.derive_private_key(3, ec.SECP256R1(), default_backend())
    three_G = private_key_3.public_key().public_numbers()
    private_key_31 = ec.derive_private_key(31, ec.SECP256R1(), default_backend())
    thirty_one_G = private_key_31.public_key().public_numbers()
    return {
        "2G": (two_G.x, two_G.y),
        "G + (2G)": (three_G.x, three_G.y), 
        "31G": (thirty_one_G.x, thirty_one_G.y)
    }

In [11]:
print("=== ecdsa库结果 ===")
ecdsa_results = verify_with_ecdsa()
for key, point in ecdsa_results.items():
    print(f"{key} = ({hex(point[0])}, {hex(point[1])})")
    
print("\n=== cryptography库结果 ===")
cryptography_results = verify_with_cryptography()
for key, point in cryptography_results.items():
    print(f"{key} = ({hex(point[0])}, {hex(point[1])})")
    
print("\n=== ecdsa 库与 cryptography 库结果比较 ===")
for key in ecdsa_results:
    ecdsa_point = ecdsa_results[key]
    crypto_point = cryptography_results[key]
    if ecdsa_point == crypto_point:
        print(f"{key}: ecdsa 库与 cryptography 库两种方法结果一致")
    else:
        print(f"{key}: ecdsa 库与 cryptography 库两种方法结果不一致")
        print(f"  ecdsa: ({hex(ecdsa_point[0])}, {hex(ecdsa_point[1])})")
        print(f"  cryptography: ({hex(crypto_point[0])}, {hex(crypto_point[1])})")
        raise Exception("ecdsa 库与 cryptography 库两种方法结果不一致，请检查代码")

std_res = ecdsa_results

=== ecdsa库结果 ===
2G = (0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978, 0x7775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1)
G + (2G) = (0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c, 0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384fb83d9a79b127a27d5032)
31G = (0x301d9e502dc7e05da85da026a7ae9aa0fac9db7d52a95b3e3e3f9aa0a1b45b8b, 0x6551b6f6b3061223e0d23c026b017d72298d9ae46887ca61d58db6aea17ee267)

=== cryptography库结果 ===
2G = (0x7cf27b188d034f7e8a52380304b51ac3c08969e277f21b35a60b48fc47669978, 0x7775510db8ed040293d9ac69f7430dbba7dade63ce982299e04b79d227873d1)
G + (2G) = (0x5ecbe4d1a6330a44c8f7ef951d4bf165e6c6b721efada985fb41661bc6e7fd6c, 0x8734640c4998ff7e374b06ce1a64a2ecd82ab036384fb83d9a79b127a27d5032)
31G = (0x301d9e502dc7e05da85da026a7ae9aa0fac9db7d52a95b3e3e3f9aa0a1b45b8b, 0x6551b6f6b3061223e0d23c026b017d72298d9ae46887ca61d58db6aea17ee267)

=== ecdsa 库与 cryptography 库结果比较 ===
2G: ecdsa 库与 cryptography 库两种方法结果一致
G + (2G): ecdsa 库与 cr

### 五、结果比较

In [12]:
ACS_N = [_2G_a, _3G_a, _31G_a]
ACS_P = [res_2G_a, res_3G_a, res_31G_a]
PCS_N = [_2G_p, _3G_p, _31G_p]
PCS_P = [res_2G_p, res_3G_p, res_31G_p]

for key, AN, AP, PN, PP in zip(std_res, ACS_N, ACS_P, PCS_N, PCS_P):
    if std_res[key] != AN:
        print(f"标准库结果与ACS_N结果不一致: {key} = {std_res[key]} != {AN}")
    if std_res[key] != AP:
        print(f"标准库结果与ACS_P结果不一致: {key} = {std_res[key]} != {AP}")
    if std_res[key] != PN:
        print(f"标准库结果与PCS_N结果不一致: {key} = {std_res[key]} != {PN}")
    if std_res[key] != PP:
        print(f"标准库结果与PCS_P结果不一致: {key} = {std_res[key]} != {PP}")
    if std_res[key] == AN == AP == PN == PP:
        print(f"{key}:\n标准库结果与所有结果一致")

2G:
标准库结果与所有结果一致
G + (2G):
标准库结果与所有结果一致
31G:
标准库结果与所有结果一致


# 测试

In [23]:
def compute_points_projective_():
    G_jacobian = (Gx, Gy, 1)
    _65537G_jacobian = point_mul_jacobian(G_jacobian, 65537)
    _65537G = jacobian_to_affine(_65537G_jacobian)
    _2_32G_jacobian = point_mul_jacobian(G_jacobian, 2**32)
    _2_32G = jacobian_to_affine(_2_32G_jacobian)
    return _65537G, _2_32G, _65537G_jacobian, _2_32G_jacobian

print("============ 普通计算方式计算结果 ============\n")
_65537G_p, _2_32G_p, _65537G_jacobian_p, _2_32G_jacobian_p = compute_points_projective_()
#print("======== 投影坐标系的结果 ========")
#print(f"2G = ({hex(_2G_jacobian_p[0])}, {hex(_2G_jacobian_p[1])}, {hex(_2G_jacobian_p[2])})")
#print(f"G + (2G) = ({hex(_3G_jacobian_p[0])}, {hex(_3G_jacobian_p[1])}, {hex(_3G_jacobian_p[2])})")
#print(f"31G = ({hex(_31G_jacobian_p[0])}, {hex(_31G_jacobian_p[1])}, {hex(_31G_jacobian_p[2])})")
#print()
print("======== 转成仿射坐标系后的结果 ========")
print(f"(65537)G = ({hex(_65537G_p[0])}, {hex(_65537G_p[1])})")
print(f"(2^32)G = ({hex(_2_32G_p[0])}, {hex(_2_32G_p[1])})")

============ 普通计算方式计算结果 ============

======== 转成仿射坐标系后的结果 ========
(65537)G = (0x1a32b7207dface203fdee0ce2a4c7e1d33d4003d227b23a5404751a0106a9bb7, 0x5b09abf820d0206b43e965bfc80eaed843f80e06c545a705f400133a6c55bce4)
(2^32)G = (0x7fe36b40af22af8921656b32262c71da1ab919365c65dfb63a5a9e22185a5943, 0xe697d45825b636249f09f40407dca6f174b3d5867b8af212d50d152c699ca101)


In [25]:
def compute_points_mont_jacobian_test_():
    mont = Montgomery_PCS(p)
    a_mont = mont.convert_to_mont(a)
    Gx_mont = mont.convert_to_mont(Gx)
    Gy_mont = mont.convert_to_mont(Gy)
    Z_mont = mont.convert_to_mont(1)
    G_mont_jacobian = (Gx_mont, Gy_mont, Z_mont)

    _65537G_mont_jacobian = point_mul_mont_jacobian(65537, G_mont_jacobian, a_mont, mont)
    _65537G = mont_jacobian_to_affine(_65537G_mont_jacobian, mont)
    _2_32G_mont_jacobian = point_mul_mont_jacobian(2**32, G_mont_jacobian, a_mont, mont)
    _2_32G = mont_jacobian_to_affine(_2_32G_mont_jacobian, mont)
    return  _65537G, _2_32G, _65537G_mont_jacobian, _2_32G_mont_jacobian

print("============ 蒙哥马利模乘计算方式计算结果 ============\n")
res_65537G_p, res_2_32G_p, res_65537G_mont_jacobian_p, res_2_32G_mont_jacobian_p = compute_points_mont_jacobian_test_()
#print("======== 投影坐标系的结果 ========")
#print(f"(65537)G = ({hex(res_65537G_mont_jacobian_p[0])}, {hex(res_65537G_mont_jacobian_p[1])}, {hex(res_65537G_mont_jacobian_p[2])})")
#print(f"(2^32)G = ({hex(res_2_32G_mont_jacobian_p[0])}, {hex(res_2_32G_mont_jacobian_p[1])}, {hex(res_2_32G_mont_jacobian_p[2])})")
#print()
print("======== 转成仿射坐标系后的结果 ========")
print(f"(65537)G = ({hex(res_65537G_p[0])}, {hex(res_65537G_p[1])})")
print(f"(2^32)G = ({hex(res_2_32G_p[0])}, {hex(res_2_32G_p[1])})")

============ 蒙哥马利模乘计算方式计算结果 ============

======== 转成仿射坐标系后的结果 ========
(65537)G = (0x1a32b7207dface203fdee0ce2a4c7e1d33d4003d227b23a5404751a0106a9bb7, 0x5b09abf820d0206b43e965bfc80eaed843f80e06c545a705f400133a6c55bce4)
(2^32)G = (0x7fe36b40af22af8921656b32262c71da1ab919365c65dfb63a5a9e22185a5943, 0xe697d45825b636249f09f40407dca6f174b3d5867b8af212d50d152c699ca101)


In [24]:
import ecdsa

def verify_with_ecdsa_():
    curve = ecdsa.ellipticcurve.CurveFp(p, a, b)
    generator = ecdsa.ellipticcurve.Point(curve, Gx, Gy, n)
    _65537G = generator * 65537
    _2_32G = generator * (2**32)
    return {
        "(65537)G": (_65537G.x(), _65537G.y()),
        "(2^32G)": (_2_32G.x(), _2_32G.y()),
    }

print("=== ecdsa库结果 ===")
ecdsa_results = verify_with_ecdsa_()
for key, point in ecdsa_results.items():
    print(f"{key} = ({hex(point[0])}, {hex(point[1])})")

=== ecdsa库结果 ===
(65537)G = (0x1a32b7207dface203fdee0ce2a4c7e1d33d4003d227b23a5404751a0106a9bb7, 0x5b09abf820d0206b43e965bfc80eaed843f80e06c545a705f400133a6c55bce4)
(2^32G) = (0x7fe36b40af22af8921656b32262c71da1ab919365c65dfb63a5a9e22185a5943, 0xe697d45825b636249f09f40407dca6f174b3d5867b8af212d50d152c699ca101)
